In [1]:
#This notebook is for testing the download of PIPS data using the web API.
#It uses urllib3 and BeautifulSoup4
import bs4
import urllib3

In [2]:
url = "http://10.163.29.26/CRD/?command=ListFiles"
http = urllib3.PoolManager()
content = http.request('GET',url)
soup = bs4.BeautifulSoup(content.data,"lxml")

In [3]:
print(soup.prettify())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
table = soup.find('table')

In [5]:
rows = table.find_all('tr')

In [6]:
headers = rows[0].find_all('td')
header = rows[0].find('td',text='Path')
header_timestamp = rows[0].find('td',text='Last Write')
print(headers)

[<td><b>Path</b></td>, <td><b>Is Directory</b></td>, <td><b>Size</b></td>, <td><b>Last Write</b></td>, <td><b>Run Now</b></td>, <td><b>Run On Power Up</b></td>, <td><b>Read Only</b></td>, <td><b>Paused</b></td>]


In [7]:
header_index = headers.index(header)
header_timestamp_index = headers.index(header_timestamp)

In [8]:
print(header,header_index)
print(header_timestamp,header_timestamp_index)

<td><b>Path</b></td> 0
<td><b>Last Write</b></td> 3


In [9]:
rows2 = rows[1:]

In [10]:
import numpy as N
import csv
from natsort import natsorted

output_dir = '/Users/dawson29/Dropbox/Projects/TriPIPS/incoming_083119/'

filelist = []
timestamp_list = []
for row in rows2:
    tokens1 = row.find_all('td')
    token = tokens1[header_index].string
    token_timestamp = tokens1[header_timestamp_index].string
    #tokens_list = tokens.strip().split(';')
    #reflectivity = N.float(tokens_list[3])
    filelist.append(token)
    timestamp_list.append(token_timestamp)

filelist_sorted = natsorted(filelist)

filelist_onesec = [x for x in filelist_sorted if 'Met' in x]
filelist_tensec = [x for x in filelist_sorted if 'Parsivel' in x]

# Output list of files and timestamps to a text file
with open(output_dir+'filelist.txt','w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(filelist_sorted, timestamp_list))

In [13]:
import os
from subprocess import call
filelist_onesec = filelist_onesec[1:]
filelist_tensec = filelist_tensec[1:]

# start_index = [i for i, x in enumerate(filelist_onesec) if '8099' in x][0]
start_index = 0
print(start_index)

filepath = filelist_onesec[start_index]
print(filepath)
drive, path = os.path.split(filepath)
print(drive)
print(path)

0
CRD/Met_Location_OneHz1293.dat
CRD
Met_Location_OneHz1293.dat


In [14]:
filepath = filelist[0]
http = urllib3.PoolManager()
url = "http://10.163.29.26/"+filepath
content = http.request('GET',url)
download = content.data
with open(output_dir+path,'wb') as output:
  output.write(download)

In [15]:
lastfilenum = start_index
filelist_onesec_todownload = filelist_onesec[lastfilenum:]
# output_dir = '/Users/dawson29/Dropbox/Projects/TriPIPS/incoming_040919/'
http = urllib3.PoolManager()
for i, filepath in enumerate(filelist_onesec_todownload):
    print("Downloading file #{0}, name: {1}".format(i, filepath))
    drive, path = os.path.split(filepath)
    url = "http://10.163.29.26/"+filepath
    content = http.request('GET',url)
    download = content.data
    with open(output_dir+path,'wb') as output:
        output.write(download)

In [16]:
# start_index = [i for i, x in enumerate(filelist_tensec) if '8099' in x][0]
start_index = 0
print(start_index)

filepath = filelist_tensec[start_index]
print(filepath)
drive, path = os.path.split(filepath)
print(drive)
print(path)

lastfilenum = start_index
filelist_tensec_todownload = filelist_tensec[lastfilenum:]
#output_dir = '/Users/dawson29/Dropbox/Projects/TriPIPS/incoming_040919/'
http = urllib3.PoolManager()
for i, filepath in enumerate(filelist_tensec_todownload):
    print("Downloading file #{0}, name: {1}".format(i, filepath))
    drive, path = os.path.split(filepath)
    url = "http://10.163.29.26/"+filepath
    content = http.request('GET',url)
    download = content.data
    with open(output_dir+path,'wb') as output:
        output.write(download)

0
CRD/Parsivel_TenHz1293.dat
CRD
Parsivel_TenHz1293.dat
